# Overfitting'i Önlemek İçin Erken Durdurma

<img src="overfit.jpg"/>
<center><b>Kaynak : https://www.veribilimiokulu.com/blog/overfitting/</b></center>

 <img src='variancebias.png'/>
 <center><b>Kaynak : https://medium.com/datarunner/bias-ve-varyans-dengesi-%C3%BCzerinden-overfitting-ve-underfitting-durumlar%C4%B1n%C4%B1n-anla%C5%9F%C4%B1lmas%C4%B1-ca26cea65bb4#:~:text=Bias%3A%20Hata%2Fyanl%C4%B1l%C4%B1k%2Fsapma,nas%C4%B1l%20yay%C4%B1ld%C4%B1%C4%9F%C4%B1n%C4%B1%20bize%20g%C3%B6steren%20de%C4%9Ferdir.</b></center>

Modelleme yapılacak veri kümesini 2 gruba ayırmakta fayda vardır.
<li>Eğitim Seti</li>
<li>Test Seti</li>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
df_iris = pd.read_csv('https://data.heatonresearch.com/data/t81-558/iris.csv', 
                na_values =[ 'NA', '?'])
df_iris.head()

,sepal_l,sepal_w,petal_l,petal_w,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
X = df_iris.iloc[:, 0:4].values
y = pd.get_dummies(df_iris['species']).values
cesitler = pd.get_dummies(df_iris['species']).columns
cesitler

Index(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype='object')

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)

In [6]:
model_iris = Sequential()
model_iris.add(Dense(50, input_dim=X.shape[1], activation='relu'))
model_iris.add(Dense(25, activation='relu'))
model_iris.add(Dense(y.shape[1], activation='softmax'))

In [7]:
model_iris.compile(loss='categorical_crossentropy', optimizer='adam')

In [8]:
monitoring = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1,
                          mode='auto', restore_best_weights=True)

In [9]:
model_iris.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=[monitoring], verbose=2, 
               epochs=1000)

Epoch 1/1000
4/4 - 0s - loss: 1.1119 - val_loss: 0.9636
Epoch 2/1000
4/4 - 0s - loss: 0.9656 - val_loss: 0.8987
Epoch 3/1000
4/4 - 0s - loss: 0.8927 - val_loss: 0.8715
Epoch 4/1000
4/4 - 0s - loss: 0.8470 - val_loss: 0.8562
Epoch 5/1000
4/4 - 0s - loss: 0.8043 - val_loss: 0.8259
Epoch 6/1000
4/4 - 0s - loss: 0.7602 - val_loss: 0.7809
Epoch 7/1000
4/4 - 0s - loss: 0.7128 - val_loss: 0.7415
Epoch 8/1000
4/4 - 0s - loss: 0.6751 - val_loss: 0.7057
Epoch 9/1000
4/4 - 0s - loss: 0.6424 - val_loss: 0.6762
Epoch 10/1000
4/4 - 0s - loss: 0.6123 - val_loss: 0.6507
Epoch 11/1000
4/4 - 0s - loss: 0.5833 - val_loss: 0.6251
Epoch 12/1000
4/4 - 0s - loss: 0.5569 - val_loss: 0.6078
Epoch 13/1000
4/4 - 0s - loss: 0.5344 - val_loss: 0.5868
Epoch 14/1000
4/4 - 0s - loss: 0.5130 - val_loss: 0.5644
Epoch 15/1000
4/4 - 0s - loss: 0.4926 - val_loss: 0.5429
Epoch 16/1000
4/4 - 0s - loss: 0.4744 - val_loss: 0.5240
Epoch 17/1000
4/4 - 0s - loss: 0.4575 - val_loss: 0.5022
Epoch 18/1000
4/4 - 0s - loss: 0.4415 - 

<b>monitor :</b> İzlenecek nicelik

<b> min_delta : </b> Bu değer küçük tutulmalıdır. İyileştirme olarak nitelendirilmesi için izlenen miktardaki minimum değişiklik, yani min_delta'dan daha düşük bir mutlak değişiklik, iyileşme yok sayılacaktır.</br>

<b> patience :</b> İyileştirme olmayan kaç epoch sonunda eğitim durudurulacak. </br>

<b> mode :</b> "auto", "min", "max" değerlerinden birini alabilir. "min" modunda, izlenen miktar azalmayı bıraktığında eğitim duracaktır; "max" modunda, izlenen miktar artmayı durdurduğunda duracaktır; "otomatik" modda, yön, izlenen miktarın adından otomatik olarak çıkarılır. </br>

<b>restore_best_weights : </b> Her zaman True olarak belirlenmelidir. İzlenen miktarın en iyi değeriyle ilgili epoch'tan geri yüklenip geri yüklenmeyeceğini belirlemeyi sağlar. False ise eğitimin son aşamasında elde edilen model ağırlıkları kullanılır.

In [10]:
from sklearn.metrics import accuracy_score
import numpy as np

In [14]:
prediction = model_iris.predict(X_test)
prediction_classes = np.argmax(prediction, axis=1)
real_classes = np.argmax(y_test, axis=1)
correction_rate = accuracy_score(real_classes, prediction_classes)
correction_rate

1.0

# Network'ten Ağırlıkları Çıkarma ve Manuel Sonuç Hesaplama

In [18]:
# Weight Initalization

Bir sinir ağının ağırlıkları, sinir ağı için çıktıyı belirlemektedir. Eğitim süreci bu ağırlıkların uygun 
olmasını sağlar. Sinir ağları eğitime ağırlıkları, rastgele bir şekilde belirleyerek başlar. Daha sonra
ağırlıklar her epochda update edilir.</b>

Bir sinir ağının rastgele olarak belirlenen ağırlıkları, o ağın ne kadar iyi eğitilebileceğini etkiler. Bir sinir ağı iyi eğitilemiyorsa, yeni bir rastgele ağırlık setiyle eğitmek bir çözüm olarak denenebilir. Bununla birlikte, bir sinir ağının mimarisini farklı gizli katman ve nöron kombinasyonları ile denemek çok uzun zaman alır ve yorucu olur. Yeni bir katman eklenmesi ve ağın performansın iyileşmesi durumunda, bu iyileşmenin yeni katmandan mı yoksa yeni ağırlık kümesinden mi kaynaklandığı araştırılmalıdır. Bu belirsizlik nedeniyle, ağırlık başlatma algoritmasında iki temel soruya cevap aranır:</br>

<li>Algoritma ne kadar tutarlı bir şekilde iyi ağırlıklar sağlıyor?</li>
<li>Algoritmanın sağladığı ağırlıklar ne kadar avantaj sağlıyor?</li>

Ağırlık başlatmaya yönelik en yaygın, ancak en az etkili yaklaşımlardan biri, ağırlıkları belirli bir aralıktaki rastgele değerlere ayarlamaktır. Genellikle -1 ile +1 veya -5 ile +5 arasındaki sayılar seçilir. Her seferinde aynı rastgele ağırlık setinin kullanıldığından emin olunmak isteniorsa, seed değeri kullanılmalıdır. </br>

Rastgele ağırlık başlatma ile ilgili bir problem, bazı seed değerleri tarafından oluşturulan rastgele ağırlıkların eğitilmesinin diğerlerinden çok daha zor olmasıdır. Bazı durumlarda, ağırlıklar o kadar kötü olabilir ki eğitim imkansızdır. Belirli bir ağırlık setiyle bir sinir ağının eğitilemeyeceği fark edilirse, farklı bir seed değeri kullanılarak yeni bir ağırlık seti oluşturulmalıdır.</br>

Weight Initialization problemi için çeşitli araştırmalar yapılmaktadır. Bu araştırmalardan biri (Glorot, X., and Bengio, Y. Understanding the difficulty of training deep feedforward neural net-
works. In Proceedings of the thirteenth international conference on artificial intelligence and statistics
(2010), pp. 249–256.) çalışmada tanıtılan Xavier weight initialization algoritmasıdır. Bu algoritma, kabul edilebilir tutarlılıkta iyi ağırlıklar üretmektedir ve normal olarak dağıtılmış rasgele sayılar kullanır.</br>

Xavier weight initialization algoritmasını kullanırken, normal olarak dağılan rasgele sayıların, çoğu programlama dilinin ürettiği 0 ile 1 arasındaki tipik rasgele sayılar olmadığını bilmek gerekmektedir.Normal dağılımda üretilen sayılar ortalamanın minimum -3 standart sapma altında veya maksimum +3 standart sapma üstünde olabilir. </br>


<img src='normaldist.png'/>
<center><b>Kaynak : https://aakinshin.net/posts/normality/</b></center>



 Xavier weight initialization tüm ağırlıkları normal olarak dağıtılmış rastgele sayılara ayarlar. Bu ağırlıklar ortalaması her zaman 0'dır. Ancak standart sapma, mevcut ağırlık katmanı için kaç bağlantı bulunduğuna bağlı olarak değişir. Aşağıdaki denklemle hesaplanabilir.
 
 <img src='variance.png'/>
 
Yukarıdaki denklem, tüm ağırlıklar için varyansın nasıl elde edileceğini gösterir. Varyansın karekökü standart sapmadır.Bu işlem sinir ağındaki her katman için tamamlanır.

<img src='examplevar.png'/>
<center><b>Kaynak : Jeff Heaton, Applications of Deep Neural Networks</b></center>

# XOR Probleminin NN ile Çözümü

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

In [16]:
X = np.array([[0, 0], [1, 0], [0, 1], [1, 1]])

In [17]:
y = np.array([0, 1, 1, 0])

In [1]:
done = False
epochs = 1

while not done:
    print(f"{epochs}. iteration")
    epochs += 1
    model_xor = Sequential()
    model_xor.add(Dense(2, input_dim=2, activation='relu'))
    model_xor.add(Dense(1, activation='linear'))
    model_xor.compile(loss='mean_squared_error', optimizer='adam')
    model_xor.fit(X, y, verbose=0, epochs=10000)
    
    predict = model_xor.predict(X)
    done = predict[0] < 0.01 and predict[3] < 0.01 and predict[1] > 0.9 and predict[2] > 0.9
    print(predict)

1. iteration


NameError: name 'Sequential' is not defined

In [8]:
# ağılıklar alınıyor
for layer_number, layer in enumerate(model_xor.layers):
    weights = layer.get_weights()[0]
    biases = layer.get_weights()[1]
    
    for to_neuron_number, bias in enumerate(biases):
        print(f'{layer_number}B ->\t L{layer_number + 1}N{to_neuron_number}:{bias}')
    
    for from_neuron_number, weight in enumerate(weights):
        for to_neuron_number, weight2 in enumerate(weight):
            print(f'L{layer_number}N{from_neuron_number}\t -> L{layer_number + 1}N{to_neuron_number} = {weight2}')

0B ->	 L1N0:-9.16545204177055e-08
0B ->	 L1N1:-1.6095487609391057e-08
L0N0	 -> L1N0 = -0.8739746809005737
L0N0	 -> L1N1 = 1.2201192378997803
L0N1	 -> L1N0 = 0.8739748001098633
L0N1	 -> L1N1 = -1.2201192378997803
1B ->	 L2N0:-8.811766871730242e-10
L1N0	 -> L2N0 = 1.1441978216171265
L1N1	 -> L2N0 = 0.8195920586585999


Model her çalıştırıldığında farklı ağırlıklar elde edilir. Yani XOR problemini çözmenin farklı yolları vardır.   
<b>Aşağıda bu ağırlıklar kullanılarak elle nasıl hesaplama yapıalbileceği gösterilmiştir</b>

In [20]:
input0 = 0
input1 = 1

hidden0sum = (input0 * -0.87) + (input1 * 0.87) + (-9.16e-8)
hidden1sum = (input0 * 1.22) + (input1 * -1.22) + (-1.60e-8)
print(hidden0sum)
print(hidden1sum)

hidden0 = max(0, hidden0sum)
hidden1 = max(0, hidden1sum)
print(hidden0)
print(hidden1)

outputsum = (hidden0 * 1.14) + (hidden1 * 0.81) + (-8.81e-10)
print(outputsum)

output = outputsum

print(output)

0.8699999084
-1.220000016
0.8699999084
0
0.9917998946949999
0.9917998946949999


# Özellik Vektörlerinin (Feature Vector) Encode Edilmesi

Bu bölümde excel'den de aşina olunan tablo verileri ile işlemler yapmaya çalışılacaktır. Yapay sinir ağları sayısal veri kullanırlar. Bu sayısal from feature vector olarak adlandırılır. Her satır bir vektör haline gelir. Giriş nöronlarının her biri, bu vektörden bir feature yani sütun alır.

In [21]:
import pandas as pd

In [22]:
pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 5)

In [23]:
df = pd.read_csv('https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv', 
                na_values =[ 'NA', '?'])
df

,id,job,area,...,retail_dense,crime,product
0,1,vv,c,...,0.492126,0.071100,b
1,2,kd,c,...,0.342520,0.400809,c
...,...,...,...,...,...,...,...
1998,1999,qp,c,...,0.598425,0.117803,c
1999,2000,pe,c,...,0.539370,0.451973,c


In [24]:
pd.set_option('display.max_columns', 9)
pd.set_option('display.max_rows', 5)
df

,id,job,area,income,...,pop_dense,retail_dense,crime,product
0,1,vv,c,50876.0,...,0.885827,0.492126,0.071100,b
1,2,kd,c,60369.0,...,0.874016,0.342520,0.400809,c
...,...,...,...,...,...,...,...,...,...
1998,1999,qp,c,67949.0,...,0.909449,0.598425,0.117803,c
1999,2000,pe,c,61467.0,...,0.925197,0.539370,0.451973,c


In [25]:
df.columns

Index(['id', 'job', 'area', 'income', 'aspect', 'subscriptions',
       'dist_healthy', 'save_rate', 'dist_unhealthy', 'age', 'pop_dense',
       'retail_dense', 'crime', 'product'],
      dtype='object')

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2000 non-null   int64  
 1   job             2000 non-null   object 
 2   area            2000 non-null   object 
 3   income          1941 non-null   float64
 4   aspect          2000 non-null   float64
 5   subscriptions   2000 non-null   int64  
 6   dist_healthy    2000 non-null   float64
 7   save_rate       2000 non-null   int64  
 8   dist_unhealthy  2000 non-null   float64
 9   age             2000 non-null   int64  
 10  pop_dense       2000 non-null   float64
 11  retail_dense    2000 non-null   float64
 12  crime           2000 non-null   float64
 13  product         2000 non-null   object 
dtypes: float64(7), int64(4), object(3)
memory usage: 218.9+ KB


In [27]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,2000.0,1000.500000,577.494589,1.0,500.750000,1000.500000,1500.250000,2000.000000
income,1941.0,56924.861927,10188.938399,19846.0,51336.000000,58633.000000,64407.000000,88671.000000
...,...,...,...,...,...,...,...,...
retail_dense,2000.0,0.563258,0.152759,0.0,0.472441,0.562992,0.673228,0.972441
crime,2000.0,0.275939,0.182886,0.0,0.141712,0.241322,0.356615,0.977276


<li>Target Column yani hedef stüun tahmin edilmeye çalışılan sütundur. Bu veride hedef sütun olarak product sütunu kullanılacaktır. Yani kişinin hangi ürünü satın alacağı tahmin edilmeye çalışılacaktır</li>

<li>Varolan ID sütunu tahminelemede kullanılmayacağı için (çünkü yararlı bir bilgi içermemeketdir) silinecektir.</li>

<li>Sayısal olan alanlar üzerinde işlem yapmaya gerek olmayacaktır</li>

<li>income sütunu eksik değerler içermektedir</li>

<li>job, area ve product sürunları kategorik veri içermektedir</li>

In [28]:
df.job.unique()

array(['vv', 'kd', 'pe', '11', 'kl', 'e2', 'nb', 'al', 'de', 'dz', 'pz',
       'bf', 'f8', 'pq', 'qp', 'kp', 'am', 'po', 'nn', 'mm', 'kw', 'by',
       'rn', 'gj', 'ax', 'ob', 'qw', 'gv', 'ke', 'sa', 'ks', 'cv', 'zz'],
      dtype=object)

In [29]:
dummies = pd.get_dummies(df.job, prefix='job')
dummies.shape

(2000, 33)

In [31]:
dummies

,job_11,job_al,job_am,job_ax,...,job_rn,job_sa,job_vv,job_zz
0,0,0,0,0,...,0,0,1,0
1,0,0,0,0,...,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1998,0,0,0,0,...,0,0,0,0
1999,0,0,0,0,...,0,0,0,0


33 farklı iş kodu olduğundan, 33 dummy variable vardır. Ayrıca bir prefix belirlenmiştir, çünkü iş kodları ("ax" gibi) kendi başlarına anlamlı değillerdir.</br>

Bundan sonra elde edilen dummy variabşe'lar orjinal veriye eklenmeli ve job alanı silinmelidir.

In [32]:
df = pd.concat([df, dummies], axis = 1)
df

,id,job,area,income,...,job_rn,job_sa,job_vv,job_zz
0,1,vv,c,50876.0,...,0,0,1,0
1,2,kd,c,60369.0,...,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1998,1999,qp,c,67949.0,...,0,0,0,0
1999,2000,pe,c,61467.0,...,0,0,0,0


In [33]:
df.drop('job', axis=1, inplace=True)
df

,id,area,income,aspect,...,job_rn,job_sa,job_vv,job_zz
0,1,c,50876.0,13.100000,...,0,0,1,0
1,2,c,60369.0,18.625000,...,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1998,1999,c,67949.0,5.733333,...,0,0,0,0
1999,2000,c,61467.0,16.891667,...,0,0,0,0


In [34]:
df = pd.concat([df, pd.get_dummies(df.area, prefix='area')], axis=1)
df.drop('area', axis=1, inplace=True)

In [35]:
median = df.income.median()
df['income'] = df.income.fillna(median)

Eksik değerleri doldurmanın daha gelişmiş yolları vardır, ancak daha fazla analiz gerektirirler. Örneğin, alanların veya iş kategorilerinin her biri için bir medyan hesaplanabilir.

In [39]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,2000.0,1000.5000,577.494589,1.0,500.75,1000.5,1500.25,2000.0
income,2000.0,56975.2520,10041.612557,19846.0,51690.00,58633.0,64230.00,88671.0
...,...,...,...,...,...,...,...,...
area_c,2000.0,0.4155,0.492931,0.0,0.00,0.0,1.00,1.0
area_d,2000.0,0.0780,0.268239,0.0,0.00,0.0,0.00,1.0


In [36]:
df.columns

Index(['id', 'income', 'aspect', 'subscriptions', 'dist_healthy', 'save_rate',
       'dist_unhealthy', 'age', 'pop_dense', 'retail_dense', 'crime',
       'product', 'job_11', 'job_al', 'job_am', 'job_ax', 'job_bf', 'job_by',
       'job_cv', 'job_de', 'job_dz', 'job_e2', 'job_f8', 'job_gj', 'job_gv',
       'job_kd', 'job_ke', 'job_kl', 'job_kp', 'job_ks', 'job_kw', 'job_mm',
       'job_nb', 'job_nn', 'job_ob', 'job_pe', 'job_po', 'job_pq', 'job_pz',
       'job_qp', 'job_qw', 'job_rn', 'job_sa', 'job_vv', 'job_zz', 'area_a',
       'area_b', 'area_c', 'area_d'],
      dtype='object')

In [37]:
X_columns = df.columns.drop(['product', 'id'])

In [40]:
X_columns

Index(['income', 'aspect', 'subscriptions', 'dist_healthy', 'save_rate',
       'dist_unhealthy', 'age', 'pop_dense', 'retail_dense', 'crime', 'job_11',
       'job_al', 'job_am', 'job_ax', 'job_bf', 'job_by', 'job_cv', 'job_de',
       'job_dz', 'job_e2', 'job_f8', 'job_gj', 'job_gv', 'job_kd', 'job_ke',
       'job_kl', 'job_kp', 'job_ks', 'job_kw', 'job_mm', 'job_nb', 'job_nn',
       'job_ob', 'job_pe', 'job_po', 'job_pq', 'job_pz', 'job_qp', 'job_qw',
       'job_rn', 'job_sa', 'job_vv', 'job_zz', 'area_a', 'area_b', 'area_c',
       'area_d'],
      dtype='object')

In [41]:
X = df[X_columns].values
X

array([[5.08760000e+04, 1.31000000e+01, 1.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [6.03690000e+04, 1.86250000e+01, 2.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [5.51260000e+04, 3.47666667e+01, 1.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       ...,
       [2.85950000e+04, 3.94250000e+01, 3.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [6.79490000e+04, 5.73333333e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [6.14670000e+04, 1.68916667e+01, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

In [48]:
dummies = pd.get_dummies(df['product'])
products = dummies.columns
y = dummies.values
dummies.columns

Index(['a', 'b', 'c', 'd', 'e', 'f', 'g'], dtype='object')

Sınıflandırma sinir ağları, (2'den fazla kategori olması durumunda) categorical_crossentropy ve çıktı katmanında bir softmax aktivasyon fonksiyonu kullanmalıdır.

In [43]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [45]:
model = Sequential()
model.add(Dense(100, input_dim=X.shape[1], activation='relu', kernel_initializer='random_normal'))
model.add(Dense(50, activation='relu', kernel_initializer='random_normal'))
model.add(Dense(25, activation='relu', kernel_initializer='random_normal'))
model.add(Dense(y.shape[1], activation='softmax', kernel_initializer='random_normal'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, 
                       mode='auto', restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=[monitor], verbose=2,
         epochs=1000)


Epoch 1/1000
50/50 - 1s - loss: 5.5095 - accuracy: 0.3744 - val_loss: 1.5589 - val_accuracy: 0.5075
Epoch 2/1000
50/50 - 1s - loss: 1.3471 - accuracy: 0.4575 - val_loss: 1.2543 - val_accuracy: 0.4050
Epoch 3/1000
50/50 - 1s - loss: 1.2792 - accuracy: 0.4300 - val_loss: 1.3974 - val_accuracy: 0.4850
Epoch 4/1000
50/50 - 1s - loss: 1.2737 - accuracy: 0.4331 - val_loss: 1.2012 - val_accuracy: 0.3850
Epoch 5/1000
50/50 - 1s - loss: 1.2258 - accuracy: 0.4544 - val_loss: 1.1733 - val_accuracy: 0.5525
Epoch 6/1000
50/50 - 1s - loss: 1.2793 - accuracy: 0.4556 - val_loss: 1.1563 - val_accuracy: 0.4850
Epoch 7/1000
50/50 - 1s - loss: 1.3023 - accuracy: 0.4313 - val_loss: 1.1663 - val_accuracy: 0.4850
Epoch 8/1000
50/50 - 1s - loss: 1.2838 - accuracy: 0.4363 - val_loss: 1.1506 - val_accuracy: 0.6200
Epoch 9/1000
50/50 - 1s - loss: 1.2054 - accuracy: 0.4487 - val_loss: 1.3056 - val_accuracy: 0.3850
Epoch 10/1000
50/50 - 1s - loss: 1.2007 - accuracy: 0.4544 - val_loss: 1.1309 - val_accuracy: 0.4850

In [46]:
prediction = model.predict(X_test)
prediction = np.argmax(prediction, axis=1)

In [47]:
from sklearn.metrics import accuracy_score

accuracy_score(np.argmax(y_test, axis=1), prediction)

0.485